In [1]:
cd set3

/media/datastorage/Phong/ufc101/set3


In [2]:
ls -l

total 54776516
-rw-rw-r-- 1 bribeiro bribeiro        117 fev  2 17:20 checkpoint
-rw-rw-r-- 1 bribeiro bribeiro  109504458 fev  1 01:49 DenseNet121_CNNRNN2-set3-seq40.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro        836 fev  1 01:49 DenseNet121_CNNRNN2-set3-seq40.index
-rw-rw-r-- 1 bribeiro bribeiro      29696 fev  1 00:46 DenseNet121_CNNRNN_test_40_labels_set3.npy
-rw-rw-r-- 1 bribeiro bribeiro  605700728 fev  1 00:46 DenseNet121_CNNRNN_test_40_set3.pickle
-rw-rw-r-- 1 bribeiro bribeiro      77120 fev  1 00:46 DenseNet121_CNNRNN_train_40_labels_set3.npy
-rw-rw-r-- 1 bribeiro bribeiro 1577181368 fev  1 00:45 DenseNet121_CNNRNN_train_40_set3.pickle
-rw-rw-r-- 1 bribeiro bribeiro  138864586 jan 22 00:43 DenseNet201_CNNRNN2-set3.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro        836 jan 22 00:43 DenseNet201_CNNRNN2-set3.index
-rw-rw-r-- 1 bribeiro bribeiro  138864569 fev  1 22:19 DenseNet201_CNNRNN2-set3-seq40.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro        836 f

In [3]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [4]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

MAX_SEQ_LENGTH = 40
NUM_FEATURES = 2048

In [5]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 9624
Total videos for testing: 3696


,video_name,tag
3926,v_HorseRace_g10_c02.avi,HorseRace
3217,v_Haircut_g02_c04.avi,Haircut
4107,v_HulaHoop_g04_c03.avi,HulaHoop
8184,v_StillRings_g05_c03.avi,StillRings
5270,v_MoppingFloor_g08_c04.avi,MoppingFloor
516,v_BandMarching_g09_c01.avi,BandMarching
356,v_BabyCrawling_g13_c01.avi,BabyCrawling
1530,v_Bowling_g13_c05.avi,Bowling
9507,v_WritingOnBoard_g22_c04.avi,WritingOnBoard
155,v_ApplyLipstick_g11_c04.avi,ApplyLipstick


In [6]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)


In [7]:

def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

2023-02-01 22:51:45.307112: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 22:51:45.854195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1


In [7]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'HammerThrow', 'Hammering', 'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls', 'JumpRope', 'JumpingJack', 'Kayaking', 'Knitting', 'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'P

2023-02-02 18:05:01.654313: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 18:05:02.162543: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1


In [9]:
from tqdm import tqdm

def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in tqdm(enumerate(video_paths)):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


# train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

# print(f"Frame features in train set: {train_data[0].shape}")
# print(f"Frame masks in train set: {train_data[1].shape}")

0it [00:00, ?it/s]2023-02-01 22:51:51.848088: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2023-02-01 22:51:52.117913: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-01 22:51:52.118332: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-01 22:51:52.118366: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-01 22:51:52.120399: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-01 22:51:52.120464: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
3696it [2:25:06,  2.36s/it]


In [10]:
train_data, train_labels = prepare_all_videos(train_df, "train")

9624it [6:27:07,  2.41s/it]


In [ ]:
# # Utility for our sequence model.
# # Bidirectional

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5))(#16, 512
#         frame_features_input, mask=mask_input
#     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.LeakyReLU()(x)
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
#     return rnn_model


# # Utility for running experiments.
# def run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    
#     history = seq_model.fit(
#         [train_data[0], train_data[1]],
#         train_labels,
#         validation_split=0.2,
#         epochs=EPOCHS,
#         callbacks=[checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# _, sequence_model = run_experiment()

In [ ]:
#40.13% LSTM(2048,dropout=0.1)
#46.47% LSTM(2048,dropout=0.5)
#43.96% LSTM(4096,dropout=0.5)

In [ ]:
# class_vocab = label_processor.get_vocabulary()

# frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
# mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# # Refer to the following tutorial to understand the significance of using `mask`:
# # https://keras.io/api/layers/recurrent_layers/gru/
# x = keras.layers.LSTM(2048, return_sequences=True,dropout=0.1)(#16, 512
#     frame_features_input, mask=mask_input
# )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
# x = keras.layers.Dense(1024, activation="relu")(x)#8, 256
# output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

# rnn_model = keras.Model([frame_features_input, mask_input], output)

# rnn_model.compile(
#     loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# )

# print(rnn_model.summary())

In [ ]:
# class_vocab = label_processor.get_vocabulary()

# frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
# mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# # Refer to the following tutorial to understand the significance of using `mask`:
# # https://keras.io/api/layers/recurrent_layers/gru/
# x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.1)(#16, 512
#     frame_features_input, mask=mask_input
# )
# # x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
# x = keras.layers.Dense(1024, activation="relu")(x)#8, 256
# output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

# rnn_model = keras.Model([frame_features_input, mask_input], output)

# rnn_model.compile(
#     loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# )

# print(rnn_model.summary())

In [ ]:

# def prepare_single_video(frames):
#     frames = frames[None, ...]
#     frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
#     frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

#     for i, batch in enumerate(frames):
#         video_length = batch.shape[0]
#         length = min(MAX_SEQ_LENGTH, video_length)
#         for j in range(length):
#             frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
#         frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

#     return frame_features, frame_mask


# def sequence_prediction(path):
#     class_vocab = label_processor.get_vocabulary()

#     frames = load_video(os.path.join("test", path))
#     frame_features, frame_mask = prepare_single_video(frames)
#     probabilities = sequence_model.predict([frame_features, frame_mask])[0]

#     for i in np.argsort(probabilities)[::-1]:
#         print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
#     return frames


# # This utility is for visualization.
# # Referenced from:
# # https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# def to_gif(images):
#     converted_images = images.astype(np.uint8)
#     imageio.mimsave("animation.gif", converted_images, fps=10)
#     return embed.embed_file("animation.gif")


# test_video = np.random.choice(test_df["video_name"].values.tolist())
# print(f"Test video path: {test_video}")
# test_frames = sequence_prediction(test_video)
# to_gif(test_frames[:MAX_SEQ_LENGTH])

In [ ]:
ls -l

In [ ]:
# import numpy as np
# import tensorflow as tf 

# np.save('Inception_CNNRNN_train_data.npy', train_data)    # .npy extension is added if not given
# # np.save('Inception_CNNRNN_test_data.npy', test_data[0])    # .npy extension is added if not given
# # np.save('Inception_CNNRNN_train_labels.npy', train_data[1])    # .npy extension is added if not given
# # np.save('Inception_CNNRNN_test_labels.npy', test_labels)    # .npy extension is added if not given

In [ ]:
# print(train_data[0].shape)
# print(train_data[1].shape)

In [ ]:
# print(train_data.type)

In [17]:
import pickle

with open('InceptionV3_CNNRNN_train_40_set3.pickle', 'wb') as f:
    pickle.dump(train_data, f)

In [18]:
import pickle

with open('InceptionV3_CNNRNN_test_40_set3.pickle', 'wb') as f:
    pickle.dump(test_data, f)

In [19]:
import numpy as np
import tensorflow as tf 

# np.save('densenet_train_data.npy', train_data)    # .npy extension is added if not given
# np.save('densenet_test_data.npy', test_data)    # .npy extension is added if not given
np.save('InceptionV3_CNNRNN_train_40_labels_set3.npy', train_labels)    # .npy extension is added if not given
np.save('InceptionV3_CNNRNN_test_40_labels_set3.npy', test_labels)    # .npy extension is added if not given

In [ ]:
# ls -l

In [ ]:
# mv Inception_CNNRNN_20.pickle Inception_CNNRNN_train_20.pickle

In [8]:
import numpy as np
import tensorflow as tf 
import pickle

with open('InceptionV3_CNNRNN_train_40_set3.pickle', 'rb') as f:
     d_train_data = pickle.load(f)
with open('InceptionV3_CNNRNN_test_40_set3.pickle', 'rb') as f:
     d_test_data = pickle.load(f)
        
# d_train_data = np.load('densenet_train_data.npy')
# d_test_data = np.load('densenet_test_data.npy')
d_train_labels = np.load('InceptionV3_CNNRNN_train_40_labels_set3.npy')
d_test_labels = np.load('InceptionV3_CNNRNN_test_40_labels_set3.npy')

In [10]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
# print(label_processor.get_vocabulary())

def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
        frame_features_input, mask=mask_input
    )    
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
#         frame_features_input, mask=mask_input
#     )
#     x = keras.layers.LSTM(2048)(x)#8, 256
#     x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(1024)(x)#8, 256
    x = keras.layers.GaussianNoise(0.2)(x)
    x = keras.layers.LeakyReLU(0.1)(x)    
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
    return rnn_model

# Utility for running experiments.
def d_run_experiment():
    filepath = "InceptionV3_CNNRNN2-set3-seq40"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    print(seq_model.summary())
    
    optimizer = keras.optimizers.SGD(lr=1e-2)
#     optimizer = keras.optimizers.Adadelta()
    
    seq_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )    
    
    history = seq_model.fit(
        [d_train_data[0], d_train_data[1]],
        d_train_labels,
#         validation_split=0.2,
        validation_data=([d_test_data[0], d_test_data[1]],
        d_test_labels),
        epochs=100,
#         epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([d_test_data[0], d_test_data[1]], d_test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


d_run_experiment()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 40, 2048)]   0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 40)]         0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 2048)         33562624    ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 dense_2 (Dense)                (None, 1024)         2098176     ['lstm_1[0][0]']           

300/301 [============================>.] - ETA: 0s - loss: 0.0036 - accuracy: 1.0000
Epoch 21: val_loss did not improve from 1.12115
301/301 [==============================] - 19s 62ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 1.1995 - val_accuracy: 0.7324
Epoch 22/100
300/301 [============================>.] - ETA: 0s - loss: 0.0033 - accuracy: 1.0000
Epoch 22: val_loss did not improve from 1.12115
301/301 [==============================] - 19s 62ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 1.1988 - val_accuracy: 0.7330
Epoch 23/100
300/301 [============================>.] - ETA: 0s - loss: 0.0031 - accuracy: 1.0000
Epoch 23: val_loss did not improve from 1.12115
301/301 [==============================] - 19s 62ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 1.2044 - val_accuracy: 0.7340
Epoch 24/100
300/301 [============================>.] - ETA: 0s - loss: 0.0029 - accuracy: 1.0000
Epoch 24: val_loss did not improve from 1.12115
301/301 [=====================

301/301 [==============================] - 19s 62ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 1.2660 - val_accuracy: 0.7321
Epoch 51/100
300/301 [============================>.] - ETA: 0s - loss: 0.0011 - accuracy: 1.0000
Epoch 51: val_loss did not improve from 1.12115
301/301 [==============================] - 19s 62ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.2671 - val_accuracy: 0.7327
Epoch 52/100
300/301 [============================>.] - ETA: 0s - loss: 0.0011 - accuracy: 1.0000
Epoch 52: val_loss did not improve from 1.12115
301/301 [==============================] - 19s 62ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.2659 - val_accuracy: 0.7335
Epoch 53/100
300/301 [============================>.] - ETA: 0s - loss: 0.0011 - accuracy: 1.0000
Epoch 53: val_loss did not improve from 1.12115
301/301 [==============================] - 19s 62ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.2684 - val_accuracy: 0.7319
Epoch 54/100
300/301 [========

300/301 [============================>.] - ETA: 0s - loss: 6.9739e-04 - accuracy: 1.0000
Epoch 79: val_loss did not improve from 1.12115
301/301 [==============================] - 19s 62ms/step - loss: 6.9729e-04 - accuracy: 1.0000 - val_loss: 1.3052 - val_accuracy: 0.7313
Epoch 80/100
300/301 [============================>.] - ETA: 0s - loss: 6.6800e-04 - accuracy: 1.0000
Epoch 80: val_loss did not improve from 1.12115
301/301 [==============================] - 19s 62ms/step - loss: 6.6800e-04 - accuracy: 1.0000 - val_loss: 1.3009 - val_accuracy: 0.7313
Epoch 81/100
301/301 [==============================] - ETA: 0s - loss: 6.5087e-04 - accuracy: 1.0000
Epoch 81: val_loss did not improve from 1.12115
301/301 [==============================] - 19s 62ms/step - loss: 6.5087e-04 - accuracy: 1.0000 - val_loss: 1.3033 - val_accuracy: 0.7308
Epoch 82/100
300/301 [============================>.] - ETA: 0s - loss: 6.6130e-04 - accuracy: 1.0000
Epoch 82: val_loss did not improve from 1.12115
30

(<keras.callbacks.History at 0x7f9b98358670>,
 <keras.engine.functional.Functional at 0x7f9b980d7e80>)

In [ ]:
#lstm(2048)
#lr=0.1 = 0.7256
#lr=0.01 = 0.7397 
#lr=0.001 = 0.7229

#noise(0.7)
#lr=0.01 = 0.7332

#noise(0.2)
#lr=0.01 = 0.7359

#lstm(4096)
#lr=0.01 = 0.7351

#lstm(1024)
#lr=0.01 = 0.7227

In [ ]:
# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
# )
# # print(label_processor.get_vocabulary())

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
#         frame_features_input, mask=mask_input
#     )    
# #     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
# #         frame_features_input, mask=mask_input
# #     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.GaussianNoise(0.4)(x)
#     x = keras.layers.LeakyReLU(0.1)(x)    
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

# #     rnn_model.compile(
# #         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# #     )
#     return rnn_model

# # Utility for running experiments.
# def d_run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    

#     lr_schedule = keras.callbacks.LearningRateScheduler(
#                   lambda epoch: 1e-6 * 10**(4*epoch / 10))
    
#     optimizer = keras.optimizers.SGD(lr=1e-3)
    
#     seq_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
#     )    
# #     seq_model.compile(optimizer=optimizer,
# #                       loss='categorical_crossentropy',
# #                      metrics=['accuracy'])     
    
#     history = seq_model.fit(
#         [d_train_data[0], d_train_data[1]],
#         d_train_labels,
# #         validation_split=0.2,
#         validation_data=([d_test_data[0], d_test_data[1]],
#         d_test_labels),
#         epochs=20,
# #         epochs=EPOCHS,
#         callbacks=[lr_schedule,checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([d_test_data[0], d_test_data[1]], d_test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# his, seq = d_run_experiment()


In [ ]:
# from matplotlib import pyplot as plt

# plt.semilogx(his.history['lr'], his.history['accuracy'])
# plt.axis([1e-6, 1, 0, 1])
# plt.xlabel('lr')
# plt.ylabel('accuracy')
# plt.show()

In [ ]:
# print(d_train_data[0].shape)
# print(d_train_data[1].shape)

In [ ]:
# print(d_train_data[1][0][39])

In [10]:
import numpy as np

# set length of video sequence and indices
MAX_SEQ_LENGTH = 50#override max sequence length
#max_seq=40
# copy_indices=[0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38]
# copy_indices=[0,4,8,12,16,20,24,28,32,36]
# copy_indices=[0,5,10,15,20,25,30,35]
# copy_indices=[0,8,16,24,32]
# copy_indices=[0,19]
# copy_indices=[0]
# max_seq=100
# # copy_indices=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,
#               26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,
#               51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,
#               76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99]
copy_indices=[0,2,4,6,8,10,12,14,16,18,
              20,22,24,26,28,30,32,34,36,38,
              40,42,44,46,48,50,52,54,56,58,
              60,62,64,66,68,70,72,74,76,78,
              80,82,84,86,88,90,92,94,96,98]
# copy_indices=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95]
# copy_indices=[0,10,20,30,40,50,60,70,80,90]
# copy_indices=[0,20,40,60,80]
# copy_indices=[0,25,50,75]
# copy_indices=[0,50]
d_train_data2_1 = np.copy(d_train_data[0][:,copy_indices, ])
d_train_data2_2 = np.copy(d_train_data[1][:,copy_indices, ])
d_train_data2 = (d_train_data2_1,d_train_data2_2)

d_test_data2_1 = np.copy(d_test_data[0][:,copy_indices, ])
d_test_data2_2 = np.copy(d_test_data[1][:,copy_indices, ])
d_test_data2 = (d_test_data2_1,d_test_data2_2)
# print(d_train_data2.shape)

label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
# print(label_processor.get_vocabulary())

def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
        frame_features_input, mask=mask_input
    )    
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
#         frame_features_input, mask=mask_input
#     )
#     x = keras.layers.LSTM(2048)(x)#8, 256
#     x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(1024)(x)#8, 256
    x = keras.layers.GaussianNoise(0.4)(x)
    x = keras.layers.LeakyReLU(0.1)(x)    
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
    return rnn_model

# Utility for running experiments.
def d_run_experiment():
    filepath = "DenseNet201_CNNRNN2-set2"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    print(seq_model.summary())
    
    optimizer = keras.optimizers.SGD(lr=1e-2)
#     optimizer = keras.optimizers.Adadelta()
    
    seq_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )    
    
    history = seq_model.fit(
        [d_train_data2[0], d_train_data2[1]],
        d_train_labels,
#         validation_split=0.2,
        validation_data=([d_test_data2[0], d_test_data2[1]],
        d_test_labels),
        epochs=100,
#         epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([d_test_data2[0], d_test_data2[1]], d_test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


d_run_experiment()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 50, 1920)]   0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 2048)         32514048    ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 1024)         2098176     ['lstm[0][0]']               

/home/bribeiro/miniconda3/envs/ar/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
2023-01-22 00:47:31.329971: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3681024000 exceeds 10% of free system memory.
2023-01-22 00:47:33.792844: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3681024000 exceeds 10% of free system memory.


Epoch 1/100


2023-01-22 00:47:38.692879: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


300/300 [==============================] - ETA: 0s - loss: 3.5235 - accuracy: 0.2887
Epoch 1: val_loss improved from inf to 1.86141, saving model to DenseNet201_CNNRNN2-set2
300/300 [==============================] - 28s 80ms/step - loss: 3.5235 - accuracy: 0.2887 - val_loss: 1.8614 - val_accuracy: 0.5924
Epoch 2/100
300/300 [==============================] - ETA: 0s - loss: 1.0692 - accuracy: 0.7753
Epoch 2: val_loss improved from 1.86141 to 1.06824, saving model to DenseNet201_CNNRNN2-set2
300/300 [==============================] - 21s 71ms/step - loss: 1.0692 - accuracy: 0.7753 - val_loss: 1.0682 - val_accuracy: 0.7100
Epoch 3/100
300/300 [==============================] - ETA: 0s - loss: 0.3961 - accuracy: 0.9129
Epoch 3: val_loss improved from 1.06824 to 0.91357, saving model to DenseNet201_CNNRNN2-set2
300/300 [==============================] - 21s 71ms/step - loss: 0.3961 - accuracy: 0.9129 - val_loss: 0.9136 - val_accuracy: 0.7405
Epoch 4/100
300/300 [==========================

300/300 [==============================] - ETA: 0s - loss: 7.4980e-04 - accuracy: 1.0000
Epoch 58: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 70ms/step - loss: 7.4980e-04 - accuracy: 1.0000 - val_loss: 0.8784 - val_accuracy: 0.7973
Epoch 59/100
300/300 [==============================] - ETA: 0s - loss: 7.2513e-04 - accuracy: 1.0000
Epoch 59: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 70ms/step - loss: 7.2513e-04 - accuracy: 1.0000 - val_loss: 0.8852 - val_accuracy: 0.7978
Epoch 60/100
300/300 [==============================] - ETA: 0s - loss: 7.2095e-04 - accuracy: 1.0000
Epoch 60: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 70ms/step - loss: 7.2095e-04 - accuracy: 1.0000 - val_loss: 0.8851 - val_accuracy: 0.7981
Epoch 61/100
300/300 [==============================] - ETA: 0s - loss: 6.9554e-04 - accuracy: 1.0000
Epoch 61: val_loss did not improve from 0.80296
30

Epoch 87/100
300/300 [==============================] - ETA: 0s - loss: 4.7854e-04 - accuracy: 1.0000
Epoch 87: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 71ms/step - loss: 4.7854e-04 - accuracy: 1.0000 - val_loss: 0.9011 - val_accuracy: 0.7983
Epoch 88/100
300/300 [==============================] - ETA: 0s - loss: 4.5107e-04 - accuracy: 1.0000
Epoch 88: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 71ms/step - loss: 4.5107e-04 - accuracy: 1.0000 - val_loss: 0.9018 - val_accuracy: 0.7975
Epoch 89/100
300/300 [==============================] - ETA: 0s - loss: 4.4424e-04 - accuracy: 1.0000
Epoch 89: val_loss did not improve from 0.80296
300/300 [==============================] - 21s 71ms/step - loss: 4.4424e-04 - accuracy: 1.0000 - val_loss: 0.9009 - val_accuracy: 0.7973
Epoch 90/100
300/300 [==============================] - ETA: 0s - loss: 4.4235e-04 - accuracy: 1.0000
Epoch 90: val_loss did not improve fr

(<keras.callbacks.History at 0x7f4b7ef3cfd0>,
 <keras.engine.functional.Functional at 0x7f4b7efde220>)

In [ ]:
#max seq = 40
#seq 1 = 75.33
#seq 2 = 78.52
#seq 5 = 79.38
#seq 8 = 79.38
#seq 10 = 79.78
#seq 20 = 78.68

#max seq = 100
#seq 2 = 78.79
#seq 4 = 80.34
#seq 5 = 80.88
#seq 10 = 80.40
#seq 20 = 81.12
#seq 50 = 80.02